In [1]:
import os

# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

In [2]:
%pip install haystack-ai
%pip install ragas-haystack
%pip install nltk
%pip install markdown-it-py
%pip install mdit_plain
%pip install openai
%pip install pandas
%pip install ragas-haystack
%pip install sentence-transformers
%pip install hf_xet
%pip install ollama-haystack==2.4.2
%pip install tqdm # For Progress Bar

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pipelines.markdown_pipeline import pipeline as markdown_pipeline
from pipelines.preprocessing_pipeline import pipeline as preprocessing_pipeline
from haystack.dataclasses import ByteStream, ChatMessage
from utils.markdown_utils import for_each_markdown_file
import pandas as pd
import uuid
from haystack_integrations.components.evaluators.ragas import RagasEvaluator
from tqdm import tqdm
import importlib
tqdm.pandas()

In [3]:
from config.secret import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

os.environ["EMBEDDING_MODEL_NAME"] = "Linq-AI-Research/Linq-Embed-Mistral"
os.environ["SENTENCE_TRANSFORMERS_HOME"] = "./model-assets/sentence-transformers"

os.environ["LLM_NAME"] = "gemma3:12b"
os.environ["LLM_CONTEXT_SIZE"] = "8192"

## Test Data

### Preprocessing

In [9]:

data = []

def preprocess_markdown(filename: str, bytes):
    bytestream = ByteStream(data=bytes)
    file_id = str(uuid.uuid4())
    md_documents = markdown_pipeline.run({
        "markdown_converter" : {
            "byte_streams": [bytestream],
            "id": file_id,
            "meta": {
                "title": filename
            }
        }
    })["markdown_converter"]["documents"]
    print(md_documents)
    preprocessed_documents = preprocessing_pipeline.run({
        "splitter": { "documents": md_documents }
    })["filter"]["documents"]
    data.extend([{ "document": document } for document in preprocessed_documents])

for_each_markdown_file("./data", preprocess_markdown)

df = pd.DataFrame(data)

Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 172.97it/s]


[Document(id=8a9bbbb3-4b90-42c4-b89b-8e952c67ae46, content: 'Im Verlauf der Schwangerschaft kann ein Kind mehrmals spontan seine Position ändern. Meist nimmt es ...', meta: {'title': 'Äußere Wendung.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 305.15it/s]


[Document(id=c7f93adb-f4df-4d35-b18f-dba3088e0081, content: 'Es gibt verschiedene Situationen und Dringlichkeiten bei der Planung eines Kaiserschnitts. So kann e...', meta: {'title': 'Kaiserschnitt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 183.11it/s]


...

[Document(id=d91a12ce-5f59-49b5-978b-09ac1ac37e87, content: 'Damit Sie bei dem geplanten Eingriff keine Schmerzen verspüren,

kann eine Narkose (Allgemeinanästhe...', meta: {'title': 'Narkose.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 350.46it/s]


[Document(id=2b36cda7-d77d-4827-a654-40fa178f9c45, content: 'In folgenden Situationen ist es sinnvoll, eine Geburt einzuleiten:

nach einem Fruchtblasensprung ab...', meta: {'title': 'Geburtseinleitung.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 189.80it/s]

[Document(id=14287e7e-e2f2-4bee-af22-dcc607107532, content: 'Die Geburt ist ein natürlicher Vorgang. Die modernen geburtshilflichen Methoden, insbesondere auch d...', meta: {'title': 'Geburtshilfliche Maßnahmen.md'})]


In [10]:
len(df)

361

In [11]:
document_ids = df["document"].apply(lambda doc: doc.id)
is_unique = document_ids.is_unique

print("All document IDs are unique:", is_unique)


All document IDs are unique: True


In [12]:
import os
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Sample 10 random rows from the DataFrame
sampled_df = df # df.sample(3).copy()

# Add 'question' and 'ground_truth' columns (initially empty)
sampled_df["question"] = None
sampled_df["ground_truth"] = None

# Loop through each sampled document and generate a question & ground truth answer
for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc="Generating Q&A"):
    doc = row["document"]
    content = doc.content.strip()

    # Instruction prompt: ask the model to return a JSON with question and ground truth
    prompt = f'''
    Basierend auf dem folgenden Text:
    """
    {content}
    """
    Formuliere:
    1. Eine kurze, präzise Frage, die durch den Text beantwortet werden kann (wie von einer Patientin oder einem Patienten gestellt).
    2. Die prägnante, korrekte Antwort (ground truth) basierend auf dem Text.

    Antworte ausschließlich mit einem JSON-Objekt im Format:
    {{
    "question": "...",
    "ground_truth": "..."
    }}
    '''  
    try:
        response = client.responses.create(
            input=content,
            model="gpt-4.1-mini",
            instructions=prompt,
            max_output_tokens=200,
            temperature=0.7
        )
        # Parse JSON from the model's output
        qa = json.loads(response.output_text.strip())
        sampled_df.at[idx, "question"] = qa.get("question")
        sampled_df.at[idx, "ground_truth"] = qa.get("ground_truth")

    except Exception as e:
        print(f"Error generating QA for doc ID {doc.id}: {e}")

# Merge generated questions and answers back into the main DataFrame
# First, ensure there's a doc_id column to join on
df["doc_id"] = df["document"].apply(lambda d: d.id)
sampled_df["doc_id"] = sampled_df["document"].apply(lambda d: d.id)

df = df.drop(columns=["question", "ground_truth"], errors="ignore")
df = df.merge(
    sampled_df[["doc_id", "question", "ground_truth"]],
    on="doc_id",
    how="left",
)



Generating Q&A:   0%|          | 0/361 [00:00<?, ?it/s]

Generating Q&A:   7%|▋         | 26/361 [01:12<16:53,  3.03s/it]

Error generating QA for doc ID 22473124da79d5e23fff3dc083c962b6822519b5dad753eaae4c06e8cc37fc4f: Expecting ',' delimiter: line 3 column 734 (char 816)


Generating Q&A:  13%|█▎        | 46/361 [02:14<15:31,  2.96s/it]

Error generating QA for doc ID 872e52ad620bdecb581dbb2bad5ccda10a5be151581d8b1ee1261643606ce448: Expecting value: line 1 column 1 (char 0)


Generating Q&A:  29%|██▉       | 104/361 [06:10<10:14,  2.39s/it] 

Error generating QA for doc ID 0b4f4e57aa803287a4b6af2e5e70ba4e88ce404bde17a1ff12d335ea47e5c724: Expecting value: line 1 column 1 (char 0)


Generating Q&A:  94%|█████████▍| 339/361 [17:29<01:10,  3.20s/it]

Error generating QA for doc ID 35915f774a716c2e9842448a5f8c031367727519db08f0af10cdbeb0e41a6a6c: Unterminated string starting at: line 3 column 19 (char 172)


Generating Q&A: 100%|██████████| 361/361 [18:16<00:00,  3.04s/it]


In [13]:
df

,document,doc_id,question,ground_truth
0,Document(id=5269e3d9e080caa0da06926ec80ebd8459...,5269e3d9e080caa0da06926ec80ebd84595a6d22bbab45...,Welche Position nimmt das Kind meist vor der G...,"Das Kind nimmt meist die Schädellage ein, bei ..."
1,Document(id=4da18bc9220a6f5b9d93896dcb8b855d08...,4da18bc9220a6f5b9d93896dcb8b855d084cd1d7694e34...,Was ist eine äußere Wendung und wann wird sie ...,"Eine äußere Wendung ist ein Verfahren, bei dem..."
2,Document(id=66ce999cda5155be3bd6f67dea1e3773d1...,66ce999cda5155be3bd6f67dea1e3773d1e010d8c6330f...,"Welche alternativen Methoden gibt es, um das K...",Zu den alternativen Methoden zählen Hitzestimu...
3,Document(id=46d10b6404ad4d29b97102fc444a0dfa53...,46d10b6404ad4d29b97102fc444a0dfa531437d26235c9...,Wie wird die äußere Wendung durchgeführt und w...,Die äußere Wendung wird normalerweise ambulant...
4,Document(id=18a8a2f0972f916edf320b036d8e93cd6a...,18a8a2f0972f916edf320b036d8e93cd6aa7075897b4f9...,"Welche Untersuchungen werden durchgeführt, um ...",Es werden eine Ultraschalluntersuchung zur Fes...
...,...,...,...,...
356,Document(id=629aefc575018650d75da8ab29e9697ae0...,629aefc575018650d75da8ab29e9697ae0b2069e92a7a3...,Sind Schmerzen und Missempfindungen im Bereich...,Schmerzen und Missempfindungen im Bereich des ...
357,Document(id=4dbf321558ecf0c0b58eb78c7979264d7b...,4dbf321558ecf0c0b58eb78c7979264d7b32894c19031c...,Heilt eine geringfügige Schnittverletzung bei ...,Eine geringfügige Schnittverletzung bei der Ge...
358,Document(id=5612001c9834923cc7a906a39367e140e7...,5612001c9834923cc7a906a39367e140e7bb06b4088442...,Welche gesundheitlichen Risiken bestehen für K...,Nach einem Kaiserschnitt kann es zu Anpassungs...
359,Document(id=087dbef831d59f890b6e6a5155c347aa60...,087dbef831d59f890b6e6a5155c347aa6070fea56b4f8f...,Was soll ich im Aufklärungsgespräch fragen?,Bitte fragen Sie im Aufklärungsgespräch nach a...


In [14]:
df.to_pickle("documents_with_questions.pkl")

## Evaluation

### Base RAG Evaluation

In [5]:
df = pd.read_pickle("documents_with_questions.pkl")

In [16]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

basic_indexing_pipeline = Pipeline()
basic_rag_document_store = InMemoryDocumentStore()

In [ ]:
from pipelines.indexing_pipelines.base_indexing_pipeline import add_base_indexing

add_base_indexing(basic_indexing_pipeline, basic_rag_document_store)

all_documents = df["document"].tolist()

basic_indexing_pipeline.run({
    "embedder": { 
        "documents": all_documents
    },
})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

{'writer': {'documents_written': 361}}

In [ ]:
import importlib
from ragas.metrics import ContextPrecision, AnswerCorrectness, Faithfulness
import pipelines.base_rag_pipeline
importlib.reload(pipelines.base_rag_pipeline)

base_rag_pipeline = pipelines.base_rag_pipeline.get_base_rag_pipeline(basic_rag_document_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[AnswerCorrectness(), Faithfulness(), ContextPrecision()]
)

base_rag_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
base_rag_pipeline.connect("retriever", "ragas_evaluator.documents")
base_rag_pipeline.connect("llm.replies", "ragas_evaluator.response")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> ragas_evaluator.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])
  - llm.replies -> ragas_evaluator.response (List[ChatMessage])

In [19]:
import prompt
importlib.reload(prompt)

results = []

df["answer_correctness"] = None
df["faithfulness"] = None
df["context_precision"] = None

for index, row in df[df["question"].notnull()].iterrows():
    reference = row["ground_truth"]
    question = row["question"]

    result = base_rag_pipeline.run({
        "query_embedder": { 
            "text": question,
        },
        "prompt_builder": {
            "template": [ChatMessage.from_system(prompt.PROMPT_TEMPLATE), ChatMessage.from_assistant("Hallo, wie kann ich Ihnen helfen?")] + [ChatMessage.from_user(question)]
        },
        "ragas_evaluator": {
            "query": question,
            "reference": reference,
            "reference_contexts": [row["document"].content]
        }
    })
    answer_correctness = result.get("ragas_evaluator", {}).get("result", {})["answer_correctness"]
    faithfulness = result.get("ragas_evaluator", {}).get("result", {})["faithfulness"]
    context_precision = result.get("ragas_evaluator", {}).get("result", {})["context_precision"]
    df.at[index, "answer_correctness"] = answer_correctness
    df.at[index, "faithfulness"] = faithfulness
    df.at[index, "context_precision"] = context_precision

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

Exception raised in Job[0]: AssertionError(AnswerSimilarity must be set)


In [21]:
df.to_pickle("base_rag_results.pkl")

### Context RAG Evaluation

In [4]:
df = pd.read_pickle("documents_with_questions.pkl")

In [ ]:
from pipelines.indexing_pipelines.context_indexing_pipeline import get_context_indexing_pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

context_rag_document_store = InMemoryDocumentStore()
pipeline = get_context_indexing_pipeline(context_rag_document_store)

all_documents = df["document"].tolist()

def index_with_context(filename, bytes):
    documents_from_file = [document for document in all_documents if document.meta["title"] == filename]
    file_content = bytes.decode("utf-8")
    pipeline.run({
        "contextualiser": { 
            "context": file_content,
             "documents": documents_from_file
        }
    })

for_each_markdown_file("./data", index_with_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import importlib
from ragas.metrics import ContextPrecision, AnswerCorrectness, Faithfulness
import pipelines.base_rag_pipeline
importlib.reload(pipelines.base_rag_pipeline)

base_rag_pipeline = pipelines.base_rag_pipeline.get_base_rag_pipeline(context_rag_document_store)

ragas_evaluator = RagasEvaluator(
    ragas_metrics=[AnswerCorrectness(), Faithfulness(), ContextPrecision()]
)

base_rag_pipeline.add_component(instance=ragas_evaluator, name="ragas_evaluator")
base_rag_pipeline.connect("retriever", "ragas_evaluator.documents")
base_rag_pipeline.connect("llm.replies", "ragas_evaluator.response")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
  - ragas_evaluator: RagasEvaluator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> ragas_evaluator.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])
  - llm.replies -> ragas_evaluator.response (List[ChatMessage])

In [9]:
import prompt
importlib.reload(prompt)

results = []

df["answer_correctness"] = None
df["faithfulness"] = None
df["context_precision"] = None

for index, row in df[df["question"].notnull()].iterrows():
    reference = row["ground_truth"]
    question = row["question"]

    result = base_rag_pipeline.run({
        "query_embedder": { 
            "text": question,
        },
        "prompt_builder": {
            "template": [ChatMessage.from_system(prompt.PROMPT_TEMPLATE), ChatMessage.from_assistant("Hallo, wie kann ich Ihnen helfen?")] + [ChatMessage.from_user(question)]
        },
        "ragas_evaluator": {
            "query": question,
            "reference": reference,
            "reference_contexts": [row["document"].content]
        }
    })
    answer_correctness = result.get("ragas_evaluator", {}).get("result", {})["answer_correctness"]
    faithfulness = result.get("ragas_evaluator", {}).get("result", {})["faithfulness"]
    context_precision = result.get("ragas_evaluator", {}).get("result", {})["context_precision"]
    df.at[index, "answer_correctness"] = answer_correctness
    df.at[index, "faithfulness"] = faithfulness
    df.at[index, "context_precision"] = context_precision

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
df

,document,doc_id,question,ground_truth,answer_correctness,faithfulness,context_precision
0,Document(id=5269e3d9e080caa0da06926ec80ebd8459...,5269e3d9e080caa0da06926ec80ebd84595a6d22bbab45...,Welche Position nimmt das Kind meist vor der G...,"Das Kind nimmt meist die Schädellage ein, bei ...",[0.7618610636464027],[0.8888888888888888],[0.6644444444311555]
1,Document(id=4da18bc9220a6f5b9d93896dcb8b855d08...,4da18bc9220a6f5b9d93896dcb8b855d084cd1d7694e34...,Was ist eine äußere Wendung und wann wird sie ...,"Eine äußere Wendung ist ein Verfahren, bei dem...",[0.35954921242867455],[0.95],[0.99999999999]
2,Document(id=66ce999cda5155be3bd6f67dea1e3773d1...,66ce999cda5155be3bd6f67dea1e3773d1e010d8c6330f...,"Welche alternativen Methoden gibt es, um das K...",Zu den alternativen Methoden zählen Hitzestimu...,[0.7264911556398232],[0.9285714285714286],[0.9999999999]
3,Document(id=46d10b6404ad4d29b97102fc444a0dfa53...,46d10b6404ad4d29b97102fc444a0dfa531437d26235c9...,Wie wird die äußere Wendung durchgeführt und w...,Die äußere Wendung wird normalerweise ambulant...,[0.5455391268597932],[1.0],[0.9060405643638305]
4,Document(id=18a8a2f0972f916edf320b036d8e93cd6a...,18a8a2f0972f916edf320b036d8e93cd6aa7075897b4f9...,"Welche Untersuchungen werden durchgeführt, um ...",Es werden eine Ultraschalluntersuchung zur Fes...,[0.7184585842813613],[0.8666666666666667],[0.42103174602332544]
...,...,...,...,...,...,...,...
356,Document(id=629aefc575018650d75da8ab29e9697ae0...,629aefc575018650d75da8ab29e9697ae0b2069e92a7a3...,Sind Schmerzen und Missempfindungen im Bereich...,Schmerzen und Missempfindungen im Bereich des ...,None,None,None
357,Document(id=4dbf321558ecf0c0b58eb78c7979264d7b...,4dbf321558ecf0c0b58eb78c7979264d7b32894c19031c...,Heilt eine geringfügige Schnittverletzung bei ...,Eine geringfügige Schnittverletzung bei der Ge...,None,None,None
358,Document(id=5612001c9834923cc7a906a39367e140e7...,5612001c9834923cc7a906a39367e140e7bb06b4088442...,Welche gesundheitlichen Risiken bestehen für K...,Nach einem Kaiserschnitt kann es zu Anpassungs...,None,None,None
359,Document(id=087dbef831d59f890b6e6a5155c347aa60...,087dbef831d59f890b6e6a5155c347aa6070fea56b4f8f...,Was soll ich im Aufklärungsgespräch fragen?,Bitte fragen Sie im Aufklärungsgespräch nach a...,None,None,None


In [11]:
df.to_pickle("context_rag_results.pkl")

### Context RAG + BM25

In [ ]:
df = pd.read_pickle("documents_with_questions.pkl")

In [ ]:
from pipelines.indexing_pipelines.context_indexing_pipeline import get_context_indexing_pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

context_rag_document_store = InMemoryDocumentStore()
pipeline = get_context_indexing_pipeline(context_rag_document_store)

all_documents = df["document"].tolist()

def index_with_context(filename, bytes):
    documents_from_file = [document for document in all_documents if document.meta["title"] == filename]
    file_content = bytes.decode("utf-8")
    pipeline.run({
        "contextualiser": { 
            "context": file_content,
             "documents": documents_from_file
        }
    })

for_each_markdown_file("./data", index_with_context)